## Imports

In [71]:
import pandas as pd
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np

## Connect to mongo database

In [5]:
# Connect to Mongo server

from pymongo import MongoClient

host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [6]:
# Connect to mongo db
db = client.#database name here

## Add csv data to mongo database

In [11]:
csv_dir_first_part = '/Users/philippmetzger/Documents/GitHub/BDMM/Homework 3/BDMM - 3rd Homework-20210520/csv/Project_WWI_'
table_names = ['Warehouse_Colors', 'Warehouse_PackageTypes', 'Warehouse_StockGroups', 
               'Warehouse_StockItems', 'Warehouse_StockItemStockGroups', 'Warehouse_StockItemTransactions']
warehouse_data = []
for name in table_names:
    path = csv_dir_first_part + name + '.csv'
    data = pd.read_csv(path)
    warehouse_data.append(data)

In [58]:
i = 0
warehouse_data[i]

,ColorID,ColorName
0,1,Azure
1,2,Beige
2,3,Black
3,4,Blue
4,5,Charcoal
5,6,Chartreuse
6,7,Cyan
7,8,Dark Brown
8,9,Dark Green
9,10,Fuchsia


In [50]:
for row_id in tqdm(range(warehouse_data[i].shape[0])):

    for col_id in range(warehouse_data[i].shape[1]):
        
        print(warehouse_data[i].iloc[row_id,col_id])
        
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

1
Novelty Items

2
Clothing

3
Mugs

4
T-Shirts

5
Airline Novelties

6
Computing Novelties

7
USB Novelties

8
Furry Footwear

9
Toys

10
Packaging Materials



In [100]:
# Adapted from:
# https://nicksardo.wordpress.com/2015/11/24/transferring-data-between-mysql-and-mongodb/

def add_csv(table_index):
    
    table_name = table_names[i]
    
    # Print table name
    print('Migrating table', table_name)
    
    # Get names of the columns of this table
    cols = list(warehouse_data[i].columns)
    
    # Create mongodb collection
    collection = db[table_name]
    
    # Get row count and print it
    row_count = warehouse_data[i].shape[0]
    print('Rows in this table:', str(row_count))
    
    #mongo client specifically requires python dict
    cus = dict()

    #custom record id rather than mongodb default hash id                                          
    cid = 0                                                 

    #cycle through each mySQL row
    
    #for ( row ) in tqdm(mycursor):
    for row_id in tqdm(range(warehouse_data[i].shape[0])):
        
        cid        += 1   #increment id
        cus['_id'] = cid                                    

        #check if current row is null
        for col_id in range(warehouse_data[i].shape[1]):
            
            field = warehouse_data[i].iloc[row_id,col_id]
            
            if field == None:
                #if the record is null, skip it                  
                continue
            
            else:
                row_title      = "".join( cols[col_id] )  
                
                #if isinstance(field, Decimal):
                #    field = float(str(field))
                    
                #if isinstance(field, date):
                #    field = str(field)
                
                if isinstance(field, np.int64):
                    field = int(field)
                    
                if isinstance(field, np.bool_):
                    field = bool(field)
                
                
                
                
                #add current record's field's title and value             
                cus[row_title] = field
         
        try:
            #we've completed processing this row, insert it into mongoldb      
            collection.insert_one( cus )
        except Exception as e: 
            print(e)
            print(cus)
            print()

In [97]:
i = 5
i

5

In [77]:
add_csv(i)

Migrating table Warehouse_Colors
Rows in this table: 36


  0%|          | 0/36 [00:00<?, ?it/s]

In [81]:
db[table_names[i]].count_documents({})

36

In [79]:
add_csv(i)

Migrating table Warehouse_PackageTypes
Rows in this table: 14


  0%|          | 0/14 [00:00<?, ?it/s]

In [84]:
db[table_names[i]].count_documents({})

14

In [86]:
add_csv(i)

Migrating table Warehouse_StockGroups
Rows in this table: 10


  0%|          | 0/10 [00:00<?, ?it/s]

In [87]:
db[table_names[i]].count_documents({})

10

In [92]:
add_csv(i)

Migrating table Warehouse_StockItems
Rows in this table: 227


  0%|          | 0/227 [00:00<?, ?it/s]

In [93]:
db[table_names[i]].count_documents({})

227

In [95]:
add_csv(i)

Migrating table Warehouse_StockItemStockGroups
Rows in this table: 442


  0%|          | 0/442 [00:00<?, ?it/s]

In [96]:
db[table_names[i]].count_documents({})

442

In [98]:
add_csv(i)

Migrating table Warehouse_StockItemTransactions
Rows in this table: 236667


  0%|          | 0/236667 [00:00<?, ?it/s]

In [101]:
db[table_names[i]].count_documents({})

236667

In [102]:
# Compare sql table sizes to mongo collection sizes
for i in range(len(table_names)):
    
    # Get table name
    table_name = table_names[i]
    
    # Get row count for csv
    csv_count = warehouse_data[i].shape[0]
    
    # Get row count for mongo db
    mongo_count = db[table_name].count_documents({})
    
    # Print row counts
    print(table_name)
    print('Csv row count:')
    print(csv_count)
    print('Mongo db document count:')
    print(mongo_count)

Warehouse_Colors
Csv row count:
36
Mongo db document count:
36
Warehouse_PackageTypes
Csv row count:
14
Mongo db document count:
14
Warehouse_StockGroups
Csv row count:
10
Mongo db document count:
10
Warehouse_StockItems
Csv row count:
227
Mongo db document count:
227
Warehouse_StockItemStockGroups
Csv row count:
442
Mongo db document count:
442
Warehouse_StockItemTransactions
Csv row count:
236667
Mongo db document count:
236667
